# Project: A/B Testing for ShoeFly.com

Nuestra tienda de zapatos favorita en línea, ShoeFly.com está realizando un A/B testing. Tienen dos versiones diferentes de un anuncio, que han colocado en correos electrónicos, así como en banners en Facebook, Twitter y Google. Quieren saber cómo se están desempeñando los dos anuncios en cada una de las diferentes plataformas cada día de la semana. Ayúdelos a analizar los datos usando medidas agregadas.

### Análisis de las fuentes de anuncios

1. Examine las primeras filas de `ad_clicks`.

In [1]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')

ad_clicks.head(10)

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B


2. Su administrador desea saber qué plataforma publicitaria le está dando más visitas.
¿Cuántas vistas (es decir, filas de la tabla) provienen de cada fuente `utm_source`?

In [2]:
utm_source = ad_clicks.groupby('utm_source').count().reset_index()

utm_source


,utm_source,user_id,day,ad_click_timestamp,experimental_group
0,email,255,255,80,255
1,facebook,504,504,180,504
2,google,680,680,239,680
3,twitter,215,215,66,215


3. Si la columna `ad_click_timestamp` es not null, entonces alguien hizo clic en el anuncio que se mostró.
Cree una nueva columna llamada `is_click`, que es `True` si `ad_click_timestamp` es not null y `False` de otra manera.

In [3]:
ad_clicks['is_click'] =  ad_clicks.apply(
    lambda row: False if pd.isnull(row['ad_click_timestamp']) else True,
    axis=1
)
ad_clicks



,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
...,...,...,...,...,...,...
1649,fe8b5236-78f6-4192-9da6-a76bba67cfe6,twitter,7 - Sunday,NaN,A,False
1650,fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1,facebook,5 - Friday,NaN,B,False
1651,ff3a22ff-521c-478c-87ca-7dc7b8f34372,twitter,3 - Wednesday,NaN,B,False
1652,ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732,google,1 - Monday,22:57,A,True


4. Queremos saber el porcentaje de personas que hicieron clic en los anuncios de cada `utm_source`.
Comience agrupando por `utm_source` e `is_click` y contando el número de `user_id`'s en cada uno de esos grupos. Guarde su respuesta en la variable `clicks_by_source`.

In [4]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


5. Ahora vamos a pivotar los datos para que las columnas sean `is_click` (`True` o `False`), el índice sea `utm_source`, y los valores sean `user_id`.
Guarde sus resultados en el variable `clicks_pivot`.

In [5]:
clicks_pivot = clicks_by_source.pivot(
    columns='is_click',
    index='utm_source',
    values='user_id'
    
).reset_index()

clicks_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


6. Cree una nueva columna en `clicks_pivot` llamada `percent_clicked`, que es igual al porcentaje de usuarios que han hecho clic en el anuncio desde cada `utm_source.
¿Hubo alguna diferencia en las tasas de clics para cada fuente?

In [6]:


clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


### Analizar una A/B Testing

7. La columna `experimental_group` nos dice si al usuario le fue mostrado el anuncio A o anuncio B.
¿Se mostró aproximadamente al mismo número de personas en ambos anuncios?

In [7]:
ad_clicks 



,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
...,...,...,...,...,...,...
1649,fe8b5236-78f6-4192-9da6-a76bba67cfe6,twitter,7 - Sunday,NaN,A,False
1650,fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1,facebook,5 - Friday,NaN,B,False
1651,ff3a22ff-521c-478c-87ca-7dc7b8f34372,twitter,3 - Wednesday,NaN,B,False
1652,ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732,google,1 - Monday,22:57,A,True


In [8]:
a = ad_clicks.experimental_group.str.count('A').sum()
b = ad_clicks.experimental_group.str.count('B').sum()

print(a)
print(b)


exp = ad_clicks.groupby(['experimental_group']).user_id.count().reset_index()
exp


827
827


,experimental_group,user_id
0,A,827
1,B,827


8. Usando la columna `is_click` que definimos anteriormente, comprobar si un mayor porcentaje de usuarios han hecho clic en Ad A o Ad B.

In [20]:
group = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index().pivot(
    index='experimental_group',
    columns='is_click',
    values='user_id'
).reset_index()

group

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


9. El Gerente de Producto para el A/B Test piensa que los clics podrían haber cambiado en el día de la semana.
Comience por crear dos DataFrames: `a_clicks` y `b_clicks`, que contienen sólo los resultados de grupo `A` y grupo `B`, respectivamente.

In [21]:
a_clicks = ad_clicks[ad_clicks.experimental_group == "A"]

In [22]:
a_clicks

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False
...,...,...,...,...,...,...
1643,fceb13ea-fd8c-446a-a61f-f977d404330a,twitter,6 - Saturday,NaN,A,False
1646,fd7d06ea-38b5-4ed9-acc9-777047db8c56,google,4 - Thursday,NaN,A,False
1647,fe570a20-448f-40ed-930b-8482b8a7c231,facebook,1 - Monday,20:7,A,True
1649,fe8b5236-78f6-4192-9da6-a76bba67cfe6,twitter,7 - Sunday,NaN,A,False


In [23]:
b_clicks = ad_clicks[ad_clicks.experimental_group == "B"]

In [24]:
b_clicks

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
10,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False
...,...,...,...,...,...,...
1645,fd2a5852-f0ef-4162-84a6-107a42dc46b5,twitter,3 - Wednesday,NaN,B,False
1648,fe6cfa5a-cc63-4770-8d56-c13ac8cf5bef,google,3 - Wednesday,15:6,B,True
1650,fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1,facebook,5 - Friday,NaN,B,False
1651,ff3a22ff-521c-478c-87ca-7dc7b8f34372,twitter,3 - Wednesday,NaN,B,False


10. Para cada grupo (`a_clicks` y `b_clicks`), calcule el porcentaje de usuarios que hicieron clic en el anuncio por día.

In [25]:
a_clicks_pivot = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(

    index='day',
    columns='is_click',
    values='user_id'
).reset_index()

In [26]:
a_clicks_pivot

is_click,day,False,True
0,1 - Monday,70,43
1,2 - Tuesday,76,43
2,3 - Wednesday,86,38
3,4 - Thursday,69,47
4,5 - Friday,77,51
5,6 - Saturday,73,45
6,7 - Sunday,66,43


In [28]:
a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False])

a_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


In [33]:
b_clicks_pivot= b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
    index='day',
    columns='is_click',
    values='user_id'
).reset_index()

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])

b_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


11. Compare los resultados de `A` y `B`. ¿Qué pasó en el transcurso de la semana?
¿Recomienda que su empresa utilice el anuncio `A` o el anuncio `B`?

In [34]:
#A seria mejor ==> podemos correr median o mean:

In [35]:
a_clicks_pivot.percent_clicked.median()

0.3813559322033898

In [37]:
b_clicks_pivot.percent_clicked.median()

0.296875